In [23]:
import numpy as np
import pandas as pd
import seaborn as sns
import math
import re
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.metrics import f1_score

In [2]:
path = "lab3-data.csv"
data = pd.read_csv(path,encoding = "utf-8",low_memory=False)

In [3]:
chosen_list=["EC033Q01NA","EC150Q07WA","EC150Q06WA",
             'ST005Q01TA','ST007Q01TA',"REPEAT"]
tempdf=data[chosen_list]

In [4]:
tempdf=tempdf.fillna(method='ffill')
tempdf=tempdf.fillna(method='bfill')

In [5]:
def fun1(a,b,c) :
    if a+b+c<=4 :
        return 0
    else :
        return 1
def fun2(a,b) :
    if math.sqrt((2*a**2+b**2)/3) >= 2.5 :
        return 1
    else :
        return 0
tempdf["feature_plan"]=tempdf.apply(lambda col:fun1(col['EC033Q01NA'],col['EC150Q07WA'],col['EC150Q06WA']),axis=1)
tempdf["m_d_education"]=tempdf.apply(lambda col:fun2(col['ST005Q01TA'],col['ST007Q01TA']),axis=1)
df=tempdf[["REPEAT","feature_plan"]]

In [40]:
kf = StratifiedKFold(n_splits = 5,random_state=2023,shuffle=True)
scaler = StandardScaler()
X=np.array(df["feature_plan"])
y=np.array(df["REPEAT"])
oversample = RandomOverSampler(sampling_strategy=1)
for train,valid in kf.split(X,y) :
    X_train=pd.DataFrame(df["feature_plan"][train])
    y_train=pd.DataFrame(df["REPEAT"][train])
    X_test =pd.DataFrame(df["feature_plan"][valid])
    y_test=pd.DataFrame(df["REPEAT"][valid])
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)
    #print(y_train_over)
    #print(y_train_over.value_counts())
    clf = KNeighborsClassifier(n_neighbors=2)
    clf.fit(X_train_over, np.array(y_train_over).ravel())
    y_pred = clf.predict(X_test)
    print(len(y_pred),len(y_test))
    print('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
    a1=0
    a2=0
    a3=0
    a4=0
    for i in range(len(y_test)) :
        if y_test[i]==0.0 and y_pred[i]==0.0 :
            a1+=1
        elif y_test[i]==1.0 and y_pred[i]==0.0 :
            a2+=1
        elif y_test[i]==0.0 and y_pred[i]==1.0 :
            a3+=1
        else :
            a4+=1
    print(a1,' ',a2,' ',a3,' ',a4)
    print('F1-score: %.2f' % f1_score(y_test, y_pred))

8436 8436
Accuracy: 0.76


KeyError: 0

In [166]:
np.array(df["feature_plan"])

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [29]:
  print(sum(y_pred == 1 & y_test == 1)/len(y_pred))

TypeError: Cannot perform 'rand_' with a dtyped [float64] array and scalar of type [bool]